In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency

In [2]:
pd.set_option('display.max_columns', None)

In [ ]:
# Loading the AmazonUk Price Prediction Dataset
df = pd.read_csv(r"C:\Users\dalre\Ironhack\AmazonUKdb\amz_uk_price_prediction_dataset.csv")
df

Part 1: Analyzing Best-Seller Trends Across Product Categories
Objective: Understand the relationship between product categories and their best-seller status.

Crosstab Analysis:
Create a crosstab between the product category and the isBestSeller status.
Are there categories where being a best-seller is more prevalent?
Hint: one option is to calculate the proportion of best-sellers for each category and then sort the categories based on this proportion in descending order.

In [4]:
# Generating a crosstab for 'category' and 'isBestSeller'
crosstab = pd.crosstab(df['category'], df['isBestSeller'])

crosstab

isBestSeller,False,True
category,,
3D Printers,247,1
3D Printing & Scanning,4065,2
Abrasive & Finishing Products,245,5
Action Cameras,1696,1
Adapters,251,3
...,...,...
Wind Instruments,243,7
Window Treatments,234,5
Women,17559,213


Proportion Calculation: The proportion of best-sellers in each category is calculated by dividing the number of best-sellers (True) by the total number of products in that category (True + False). This is added as a new column 'proportion_best_sellers'.

In [ ]:
# Calculate the proportion of true best-sellers for each category
#crosstab['proportion_best_sellers'] = crosstab[True] / (crosstab[True] + crosstab[False])

# # Sort the categories based on the proportion of best-sellers in descending order
# crosstab_result = crosstab.sort_values(by='proportion_best_sellers', ascending=False)

# # Display the result
# print(crosstab_result)

The provided crosstab data tells us about the prevalence of best-sellers in different product categories. Here’s a detailed interpretation:

Proportion of Best-Sellers:

The proportion_best_sellers column shows the ratio of best-sellers (True) to the total number of products in each category.
For example, the Grocery category has a proportion_best_sellers of approximately 0.058, meaning around 5.8% of grocery products are best-sellers.
Categories with Higher Prevalence of Best-Sellers:

Grocery has the highest proportion of best-sellers at 5.8135%.
Smart Home Security & Lighting follows closely with a proportion of 5.7692%.
Health & Personal Care has a proportion of 5.7686%.
These categories have the highest proportions of best-sellers, indicating that products in these categories are more likely to be best-sellers compared to others.
Categories with Lower Prevalence of Best-Sellers:

Several categories have a proportion_best_sellers of 0, indicating no products in these categories are best-sellers. Examples include Bedding Collections, CPUs, CD, Disc & Tape Players, Blank Media Cases & Wallets, and eBook Readers & Accessories.
This suggests that products in these categories are less likely to achieve best-seller status, at least within the dataset provided.
General Trends:

There is a noticeable variation in the proportion of best-sellers across different categories.
Categories related to everyday use and health, like Grocery and Health & Personal Care, have higher proportions of best-sellers.
More niche or specialized categories, like CPUs and eBook Readers & Accessories, have no best-sellers in this dataset.
Implications for Business Decisions:
Focus on High-Proportion Categories: Businesses might focus their marketing and product development efforts on categories with higher proportions of best-sellers, as these categories seem to resonate more with consumers.
Investigate Low-Proportion Categories: For categories with low or zero proportions of best-sellers, businesses could investigate potential reasons. This might involve examining product quality, customer preferences, or market trends to identify why these categories are not achieving best-seller status.
Example Summary Statement:
"The analysis of best-seller trends across product categories reveals that categories such as Grocery, Smart Home Security & Lighting, and Health & Personal Care have a higher prevalence of best-sellers, with proportions around 5.8%. In contrast, categories like Bedding Collections, CPUs, and eBook Readers & Accessories have no best-sellers, indicating a significant disparity in best-seller status across different product categories."

Statistical Tests:
Conduct a Chi-square test to determine if the best-seller distribution is independent of the product category.

In [ ]:
import math

In [5]:
# Chi-square test for 'category' and 'isBestSeller'
chi2_statistic, chi2_p_value, dof, _ = chi2_contingency(crosstab)

chi2_statistic, chi2_p_value

(36540.20270061387, 0.0)

Compute Cramér's V to understand the strength of association between best-seller status and category.

In [6]:
from scipy.stats.contingency import association
# Computing the association between variables in 'crosstab_result' using the "cramer" method
association(crosstab, method="cramer")

0.1222829439760564

#Another option to write the cramers
# Compute the total number of observations
n = crosstab_result.values.sum()

# Compute the minimum dimension of the contingency table minus 1
min_dim = min(crosstab_result.shape) - 1

# Compute Cramér's V
cramers_v = math.sqrt(chi2_statistic / (n * min_dim))
print(cramers_v)
Total Observations: The n variable is computed as the sum of all values in the contingency table.
Minimum Dimension Calculation: The min_dim variable is calculated as the minimum dimension of the contingency table minus 1.
Cramér's V Calculation: The cramers_v variable is computed using the chi-square statistic, total number of observations, and minimum dimension.

Visualize the relationship between product categories and the best-seller status using a stacked bar chart.

In [ ]:
# Plotting a stacked bar chart for the 'crosstab_result' data
crosstab.plot(kind="bar", stacked=True)

In [ ]:
top_categories = sorted_categories_by_best_seller_proportion.iloc[:10, :]

Part 2: Exploring Product Prices and Ratings Across Categories and Brands
Objective: Investigate how different product categories influence product prices.

Preliminary Step: Remove outliers in product prices.

For this purpose, we can use the IQR (Interquartile Range) method. Products priced below the first quartile minus 1.5 times the IQR or above the third quartile plus 1.5 times the IQR will be considered outliers and removed from the dataset. The next steps will be done with the dataframe without outliers.

Hint: you can check the last Check For Understanding at the end of the lesson EDA Bivariate Analysis for a hint on how to do this.

In [9]:
# Calculate IQR for price
Q1 = df['price'].quantile(0.25)
Q3 = df['price'].quantile(0.75)
IQR = Q3 - Q1

# Define outlier bounds
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Remove outliers
df_clean = df[(df['price'] >= lower_bound) & (df['price'] <= upper_bound)]

Violin Plots:

Use a violin plot to visualize the distribution of price across different product categories. Filter out the top 20 categories based on count for better visualization.
Which product category tends to have the highest median price? Don't filter here by top categories.

In [ ]:
# Filter top 20 categories based on count
top_categories = df_clean['category'].value_counts().head(20).index
df_filtered = df_clean[df_clean['category'].isin(top_categories)]

# Create a violin plot
plt.figure(figsize=(14, 8))
sns.violinplot(data=df_filtered, x='category', y='price', palette='viridis')
plt.xticks(rotation=90, ha='right')
plt.title('Distribution of Product Prices Across Top 20 Categories', fontsize=16)
plt.xlabel('Product Category', fontsize=14)
plt.ylabel('Price', fontsize=14)
plt.tight_layout()
plt.show()

# Calculate median prices
median_prices = df_clean.groupby('category')['price'].median()
highest_median_category = median_prices.idxmax()
highest_median_price = median_prices.max()

Bar Charts:

Create a bar chart comparing the average price of products for the top 10 product categories (based on count).
Which product category commands the highest average price? Don't filter here by top categories.

In [ ]:
# Calculate average prices for the top 10 categories
top_10_categories = df_clean['category'].value_counts().head(20).index
df_top10 = df_clean[df_clean['category'].isin(top_10_categories)]

# Calculate average prices
avg_prices = df_top10.groupby('category')['price'].mean().sort_values()

# Create a bar chart
plt.figure(figsize=(14, 8))
avg_prices.plot(kind='bar', color='skyblue')
plt.title('Average Price of Products in Top 10 Categories', fontsize=16)
plt.xlabel('Product Category', fontsize=14)
plt.ylabel('Average Price', fontsize=14)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# Find the category with the highest average price
print(f"highest_avg_price_category = avg_prices.idxmax()")
print(f"highest_avg_price = avg_prices.max()")




Box Plots:

Visualize the distribution of product ratings based on their category using side-by-side box plots. Filter out the top 10 categories based on count for better visualization.
Which category tends to receive the highest median rating from customers? Don't filter here by top categories.

In [ ]:
# Filter top 10 categories based on count
top_10_categories_ratings = df_clean['category'].value_counts().head(10).index
df_top10_ratings = df_clean[df_clean['category'].isin(top_10_categories_ratings)]

# Create a box plot for ratings
plt.figure(figsize=(14, 8))
sns.boxplot(data=df_top10_ratings, x='category', y='stars', palette='viridis')
plt.xticks(rotation=90, ha='right')
plt.title('Distribution of Product Ratings Across Top 10 Categories', fontsize=16)
plt.xlabel('Product Category', fontsize=14)
plt.ylabel('Rating', fontsize=14)
plt.tight_layout()
plt.show()

# Calculate median ratings
median_ratings = df_clean.groupby('category')['stars'].median()
highest_median_rating_category = median_ratings.idxmax()
highest_median_rating = median_ratings.max()

print(f"The category with the highest median rating is {highest_median_rating_category} with a median rating of {highest_median_rating:.1f}")


Part 3: Investigating the Interplay Between Product Prices and Ratings
Objective: Analyze how product ratings (stars) correlate with product prices.

In [17]:
# Calculate the correlation coefficient between price and stars
correlation = df_clean[['price', 'stars']].corr()
correlation

,price,reviews
price,1.000000,-0.008498
reviews,-0.008498,1.000000


Use a scatter plot to visualize the relationship between product rating and price. What patterns can you observe?
Use a correlation heatmap to visualize correlations between all numerical variables.
Examine if product prices typically follow a normal distribution using a QQ plot.

In [ ]:
# Scatter plot of price vs. stars
plt.figure(figsize=(14, 8))
sns.scatterplot(data=df_clean, x='price', y='stars', alpha=0.5, palette='viridis')
plt.title('Relationship Between Product Price and Rating', fontsize=16)
plt.xlabel('Price', fontsize=14)
plt.ylabel('Rating', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Correlation heatmap
plt.figure(figsize=(10, 8))
correlation_matrix = df_clean[['price', 'stars']].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=.5)
plt.title('Correlation Heatmap of Numerical Variables', fontsize=16)
plt.show()
